# Requesting Comments

1. Insert your API Key from https://console.cloud.google.com/apis/api/youtube.googleapis.com to 'DEVELOPER_KEY' in your '.env' as a string.
2. Insert your video id to 'videoId' as string. Example: https://www.youtube.com/watch?v=SIm2W9TtzR0, videoId='SIm2W9TtzR0'

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
DEVELOPER_KEY = os.getenv('DEVELOPER_KEY')

In [17]:
import googleapiclient.discovery
import googleapiclient.errors

api_service_name = 'youtube'
api_version = 'v3'

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part='snippet',
    videoId='0FtcHjI5lmw',
    maxResults=100
)
response = request.execute()
response

# for item in response['items']:
#     print(item['snippet']['topLevelComment']['snippet']['textDisplay'])

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'dVAXMKBhS7JUgenQjDxSt_t26nM',
 'pageInfo': {'totalResults': 12, 'resultsPerPage': 100},
 'items': [{'kind': 'youtube#commentThread',
   'etag': 'yIKWga1v9uMI3v21JnjeyEmUVtk',
   'id': 'UgxepygQnYgZMTkskDF4AaABAg',
   'snippet': {'channelId': 'UCaizTs-t-jXjj8H0-S3ATYA',
    'videoId': '0FtcHjI5lmw',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': 'sqE2PXfzwbxBM0YkvjS850D_zv0',
     'id': 'UgxepygQnYgZMTkskDF4AaABAg',
     'snippet': {'channelId': 'UCaizTs-t-jXjj8H0-S3ATYA',
      'videoId': '0FtcHjI5lmw',
      'textDisplay': 'This really helped me out. Thank you',
      'textOriginal': 'This really helped me out. Thank you',
      'authorDisplayName': '@JamesDeanDesigns',
      'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AIf8zZSc0bEvQkNMzOtWxrYFxxoqJyAY2uk3JN3esrUQ-g=s48-c-k-c0x00ffffff-no-rj',
      'authorChannelUrl': 'http://www.youtube.com/channel/UCzBQxkTsKbaKo1tqqOdxLOQ',
      'authorChannelId': {'va

# Requesting Comments to DataFrame

### Request for specific channel

**Imports & Youtube API Init**

In [2]:
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd

api_service_name = 'youtube'
api_version = 'v3'

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=DEVELOPER_KEY)

**Request Video Ids using Search**

In [22]:
CHANNEL_ID = ''

request_video_ids = youtube.search().list(
    part='snippet',
    channelId=CHANNEL_ID,
    maxResults=50,
    type='video',
)
response_video_ids = request_video_ids.execute()

video_ids = []
    
while True:
    for item in response_video_ids['items']:
        video_ids.append(item['id']['videoId'])
    try:
        next_page_token_search = response_video_ids['nextPageToken']
    except KeyError:
        break
    request_video_ids = youtube.search().list(
        part='snippet',
        channelId=CHANNEL_ID,
        maxResults=50,
        type='video',
        pageToken=next_page_token_search
    )
    response_video_ids = request_video_ids.execute()

video_ids

['bb1jO1ihiV0',
 'v-AARuXfjkM',
 'E-bIc7LpbT8',
 'ypWXrqVTRe0',
 'a5xRHBc289c',
 'Mw30CgaXiQw',
 'esFAPhH0sYM',
 'qk692VXpgT4',
 '239cOqTfopg',
 'Hy8s6cMpEko',
 'D-VytLhH-KE',
 'oI6aXhowFDY',
 'rg_zwK_sSEY',
 'W-w3WfgpcGg',
 'iogcY_4xGjo',
 'eC2FCTh6PCw',
 '-FyjEnoIgTM',
 'hGNrrsGKNMk',
 'B9wIV1dbxzs',
 't9VWICGOD90',
 'y_YL3R32RzU',
 'jKIEUdAMtrQ',
 '6ZUIwj3FgUY',
 'VT1-sitWRtY',
 'fwK7ggA3-bU',
 'vBAwCwLhZLc',
 'FocFked1TbQ',
 'C7uuz2l4RQY',
 'k4V3Mo61fJM',
 'rap8SoUIPaw',
 '_CXXVFPO6f0',
 'pRfmrE0ToTo',
 'CjNTUbUnwfY',
 '6Ux6SlOE9Qk',
 'bgUh1yTUeZw',
 'k8o227Jy1Sk',
 'lW9ep22YmlM',
 'DIm2cqy7-AI',
 'HRdyfERwQZs',
 'lir3dzYIhz0',
 'rJmQhKwlUJU',
 'ykuUDTyT7Gg',
 'TkV5709EG5M',
 'bleOTMDa3_4',
 '1ahQNqglBJI',
 'wv8-QnT62jE',
 'AEIVhBS6baE',
 '3M94PcUeMtY',
 'VY-Ewukt8tU',
 'TaV0lM5zb5Y',
 'wvOQ56rwyas',
 'qgy7vEje5-w',
 '5hs8cZXxJU4',
 'Lm9k6xrnLNU',
 '3DqSLbCSt-M',
 'MPST-5J-7sk',
 '3WyYvgA939g',
 'nnZpgbJQVXw',
 'AzdC3-diq00',
 'MG-bvSY2f_4',
 '_aVWFv3f_Ic',
 '0866g-nXR_Y',
 '4SxWtQ